In [ ]:
from pytorch_model import load_wpod
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torchvision import transforms, utils, models
from torch import nn
import torch
from src.label import Label, Shape
from src.utils import getWH, nms, im2single, IOU_centre_and_dims
from src.projection_utils import getRectPts, find_T_matrix
import time
from utils import *
from tqdm import tqdm

In [ ]:
pil_to_tensor = transforms.ToTensor()
tensor_to_pil = transforms.ToPILImage()

In [ ]:
class DLabel (Label):

    def __init__(self,cl,pts,prob):
        self.pts = pts
        tl = np.amin(pts,1)
        br = np.amax(pts,1)
        Label.__init__(self,cl,tl,br,prob)


def reconstruct(Iorig,I,Y,out_size,threshold=.9):

    net_stride 	= 2**4
    side = ((208. + 40.)/2.)/net_stride # 7.75
    
    Probs = Y[...,0]
    Affines = Y[...,2:]
    rx,ry = Y.shape[:2]
    ywh = Y.shape[1::-1]
    iwh = np.array(I.shape[1::-1],dtype=float).reshape((2,1))
    xx,yy = np.where(Probs>threshold)
    
    WH = getWH(I.shape)
    MN = WH/net_stride

    vxx = vyy = 0.5 #alpha

    base = lambda vx,vy: np.matrix([[-vx,-vy,1.],[vx,-vy,1.],[vx,vy,1.],[-vx,vy,1.]]).T
    labels = []

    for i in range(len(xx)):
        y,x = xx[i],yy[i]
        affine = Affines[y,x]
        prob = Probs[y,x]
        mn = np.array([float(x) + .5,float(y) + .5])
        A = np.reshape(affine,(2,3))
        A[0,0] = max(A[0,0],0.)
        A[1,1] = max(A[1,1],0.)
        pts = np.array(A*base(vxx,vyy)) #*alpha
        pts_MN_center_mn = pts*side
        pts_MN = pts_MN_center_mn + mn.reshape((2,1))

        pts_prop = pts_MN/MN.reshape((2,1))

        labels.append(DLabel(0,pts_prop,prob))

    final_labels = nms(labels,.1)
    TLps = []

    if len(final_labels):
        final_labels.sort(key=lambda x: x.prob(), reverse=True)
        for i,label in enumerate(final_labels):

            t_ptsh 	= getRectPts(0,0,out_size[0],out_size[1])
            ptsh 	= np.concatenate((label.pts*getWH(Iorig.shape).reshape((2,1)),np.ones((1,4))))
            H 		= find_T_matrix(ptsh,t_ptsh)
            Ilp 	= cv2.warpPerspective(Iorig,H,out_size,borderValue=.0)

            TLps.append(Ilp)

    return final_labels,TLps
    

def detect_lp(model,I,max_dim,net_step,out_size,threshold,masked_pattern=None,train=None):

    min_dim_img = min(I.shape[:2])
    factor 		= float(max_dim)/min_dim_img

    w,h = (np.array(I.shape[1::-1],dtype=float)*factor).astype(int).tolist()
    w += (w%net_step!=0)*(net_step - w%net_step)
    h += (h%net_step!=0)*(net_step - h%net_step)
    Iresized = cv2.resize(I,(w,h))

    Tn = Iresized.copy()
    Tn = Tn.reshape((1,Tn.shape[0],Tn.shape[1],Tn.shape[2]))
    Tn = torch.tensor(Tn, device=device).permute(0,3,1,2)
    start = time.time()
    model.eval()
    if masked_pattern is not None:
        Tn = masked_pattern*Tn[0]
    if train:
        Yr = model(Tn).permute(0,2,3,1)
        Yr = np.squeeze(Yr)
        return None,None,None,Yr,Tn
    else:
        Yr = model(Tn).permute(0,2,3,1)
        Yr = np.squeeze(Yr)
        Y2 = Yr.cpu().detach().numpy()
    elapsed = time.time() - start
    
    L,TLps = reconstruct(I,Iresized,Y2,out_size,threshold)

    return L,TLps,elapsed,Yr.cpu(),Tn


In [ ]:
def labels2output_map(label,lppts,dim,stride):

	side = ((float(dim) + 40.)/2.)/stride # 7.75 when dim = 208 and stride = 16

	outsize = int(dim/stride)
	Y  = np.zeros((outsize,outsize,2*4+1),dtype='float32')
	MN = np.array([outsize,outsize])
	WH = np.array([dim,dim],dtype=float)

	tlx,tly = np.floor(np.maximum(label.tl(),0.)*MN).astype(int).tolist()
	brx,bry = np.ceil (np.minimum(label.br(),1.)*MN).astype(int).tolist()

	for x in range(tlx,brx):
		for y in range(tly,bry):

			mn = np.array([float(x) + .5, float(y) + .5])
			iou = IOU_centre_and_dims(mn/MN,label.wh(),label.cc(),label.wh())

			if iou > .5:

				p_WH = lppts*WH.reshape((2,1))
				p_MN = p_WH/stride

				p_MN_center_mn = p_MN - mn.reshape((2,1))

				p_side = p_MN_center_mn/side

				Y[y,x,0] = 1.
				Y[y,x,1:] = p_side.T.flatten()

	return Y

In [ ]:
def adjust_pts(pts,lroi):
	return pts*lroi.wh().reshape((2,1)) + lroi.tl().reshape((2,1))


output_dir = "./temp"

lp_threshold = .5

wpod_net = load_wpod()
wpod_net.to(device)
wpod_net.eval()
print('Searching for license plates using WPOD-NET')

Ivehicle = cv2.imread('Plate_examples/lptest.jpg')
#Ivehicle = cv2.resize(Ivehicle,(208,208))
#Ivehicle = Ivehicle[450:630, 480:1170]
plt.imshow(Ivehicle)
print(Ivehicle.shape)
#Ivehicle = cv2.bitwise_and(Ivehicle, Ivehicle, mask=cv2.bitwise_not(get_mask(Ivehicle)))
ratio = float(max(Ivehicle.shape[:2]))/min(Ivehicle.shape[:2])
side  = int(ratio*288.)
bound_dim = min(side + (side%(2**4)),608)
print("\t\tBound dim: %d, ratio: %f" % (bound_dim,ratio)) 

Llp,LlpImgs,_,Y,_ = detect_lp(wpod_net,im2single(Ivehicle),bound_dim,2**4,(240,80),lp_threshold)

if len(LlpImgs):
    print('found')
    Ilp = LlpImgs[0]
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)

    s = Shape(Llp[0].pts)

    plt.imshow(Ilp)


In [ ]:
tl,br = s.pts.min(1),s.pts.max(1)
llp = Label(0,tl,br)
Y = labels2output_map(llp,s.pts,288,16)

In [ ]:
Y[...,0][np.where(Y[...,0]>0.5)]

In [ ]:
def get_mask(Ivehicle,h=None,w=None,d=None):
    if h:
        height,width,depth = h,w,d
    else: height,width,depth = Ivehicle.shape
    x = Ivehicle.copy()
    x = cv2.resize(x,(height,width))
    circle_img = np.zeros((height,width), np.uint8)
    #cv2.rectangle(circle_img,(86,107),(212,152),280,thickness=-1)
    #cv2.rectangle(circle_img,(86,107),(212,152),280,thickness=-1)
    print(circle_img.shape)
    #circle_img = circle_img.reshape((1,circle_img.shape[0],circle_img.shape[1]))
    #circle_img = torch.tensor(circle_img).permute(0,1,2)
    #cv2.rectangle(circle_img,(480,480),(1170,570),255,thickness=-1)
    plt.imshow(cv2.bitwise_and(x, x, mask=circle_img))
    return circle_img
get_mask(Ivehicle,288,288,3)

In [ ]:
account_resize = True
repeat_size=1

#For resize post convolution
if account_resize:
    repeat_size = int(3024/3024)
    img = Ivehicle.resize((208,208))
    model_img_size = img.size[0]
    img_t = pil_to_tensor(img)
    img_t = img_t.to(device)
    def resize2d(img, size):
        return (F.adaptive_avg_pool2d(Variable(img), size)).data
    def upsample2d(img, size=224):
        upsample = nn.Upsample(size=size, mode='bilinear', align_corners=False)
        return upsample(torch.unsqueeze(img, 0))[0]
else:
    model_img_size = 224

In [ ]:
model_img_size = 512

In [ ]:
# typical exposure is in form 1/n s. Available: 15, 20, 25 30, 40, 50, 60, 80, 100, 125, 160, 200, 250
exposure = 125 
exp_micros = 1000000/exposure          # get exposure in microseconds
img_ratio = 2 / model_img_size      # every row in model is img_ratio rows in original image
model_tr = 10 * img_ratio              # multiply real tr (10 micros) by img_ratio to find model tr
conv_size = exp_micros / model_tr      # divide exposure time by tr to find convolution size
conv_size = int(conv_size)             # Need closest integer approximation. Won't cause a significant difference
conv_size

In [ ]:
#Ivehicle = cv2.imread('Plate_examples/lptest.jpg')
r = cv2.imread('Plate_examples/lp_red.jpg')
r = cv2.cvtColor(r, cv2.COLOR_BGR2RGB)
g = cv2.imread('Plate_examples/lp_green.jpg')
b = cv2.imread('Plate_examples/lp_blue.jpg')
b = cv2.cvtColor(r, cv2.COLOR_BGR2RGB)
w = cv2.imread('Plate_examples/lp_white.jpg')
canvas = np.zeros([2268,4032,3],dtype=np.uint8)
canvas[:,:,0] = r[:,:,0]
canvas[:,:,1] = g[:,:,1]
canvas[:,:,2] = b[:,:,2]
Ivehicle = canvas
Ivehicle.shape
#Ivehicle = cv2.cvtColor(Ivehicle, cv2.COLOR_BGR2RGB)
#Ivehicle = cv2.resize(Ivehicle,(208,208))

In [ ]:
sz = model_img_size + conv_size - 1 # 300   #Length of input signal
c = 0    #Ambient light ratio
c_limits = [0,0]
batch = 8
channels = 3
# change of variable term to optimise on
w = torch.rand([channels,sz,1], requires_grad=True, dtype=torch.float, device=device)

#Create the mask to only illuminate the object
#mask = torch.tensor(get_mask(Ivehicle,288,288,3), dtype=torch.float, device=device)
#mask = torch.tensor(get_mask(Ivehicle,288,288,3), device=device)
mask = torch.ones([3,512,912], device=device)
mask = mask / torch.max(mask)
#mask = get_mask(Ivehicle,288,288,3)

#Target and original class labels
#orig = torch.tensor([classidx], dtype=torch.long, device=device)

#Model parameters
lr = 1e-1
n_epochs = 1000
optimizer = optim.SGD([w], lr=lr, momentum=0.9, nesterov=True)
#optimizer = optim.Adam([w], lr=lr)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
labelf = torch.flatten(torch.tensor(Y[...,0], dtype=torch.float, device=device))
labelnf = torch.flatten(torch.tensor(1-Y[...,0], dtype=torch.float, device=device))
def logloss(ptrue, pred, eps=10e-10):
    Pred = torch.clamp(pred,eps,1.)
    Pred = -torch.log(Pred)
    Pred = torch.dot(Pred,ptrue)
    #Pred = torch.reshape(Pred,(b,h*w*ch))
    #Pred = torch.sum(Pred,1)
    return Pred

In [ ]:
n_epochs = 500

In [ ]:
def train_run(update_frequency, print_frequency, n_epochs):
    #Track the loss to target and original class
    loss_list = []

    #obj_dict = {}
    #w = -1*w
    #Optimisation loop. initially untargeted
    for epoch in tqdm(range(n_epochs)):

        if channels==1:
            n_w = w.repeat(3,1,1)
        else:
            n_w = w
        #with torch.no_grad():
        #    n_w = torch.clamp(n_w, max=1)
        # For resize post convolution
        #if account_resize:
        #    n_w = torch.repeat_interleave(n_w, repeats=repeat_size, dim=1)

        sig_height = model_img_size + conv_size - 1
        gy, new_w = fttogy(n_w, batch, None, c_limits, sig_height, conv_size, 0)
        #pattern = 1-mask + torch.mul(gy,mask)
        #new_w.retain_grad()
        #gy.retain_grad()
        #with torch.autograd.set_detect_anomaly(True):
        Llp,LlpImgs,x,Yr,T = detect_lp(wpod_net,im2single(Ivehicle),bound_dim,2**4,(240,80),lp_threshold,gy,train=True)
        #Yr.retain_grad()
        #loss = -1*torch.sum(torch.log(1-Yr[...,0][np.where(Y[...,0]>0.5)]))
        loss = -1*torch.log(1-torch.max(torch.max(Yr[...,0],dim=1)[0],dim=1)[0])
        loss = torch.sum(loss)
        #loss.retain_grad()
        if epoch%print_frequency==0:
            #print(Yr[...,0][np.where(Y[...,0]>0.5)])
            loss_list.append(loss)

        if epoch%100==0:
            #print(Yr[...,0][np.where(Y[...,0]>0.5)])
            print(loss)
            
        loss.backward()   

        if epoch%update_frequency==0:
            #print('loss_grad',loss.grad)
            #print('gy_grad', gy.grad[gy.grad>0])
            #print('w_grad', w.grad)
            #print('new_w', new_w.grad[new_w.grad>0])
            optimizer.step()
            optimizer.zero_grad()
            del loss
            torch.cuda.empty_cache()
        #if epoch!=n_epochs-1:
        #    del inp
        #    del new_w
        #else:
            #saving w to be used for prediction
            #torch.save(n_w,'w_0.5_764.pt')

        #Code to check gpu allocation    
        '''
        for obj in gc.get_objects():
            try:
                if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                    #print(type(obj), obj.size())
                    if type(obj) not in obj_dict:
                        obj_dict[type(obj)] = 1
                    else:
                        obj_dict[type(obj)] += 1
            except: pass
        print(obj_dict)
        obj_dict.clear()
        '''
    return loss_list, Yr, gy, new_w, T


In [ ]:
#View original loss and target loss
update_freqs = [20]
result = []
for up in update_freqs:
    #w = torch.rand([channels,sz,1], requires_grad=True, dtype=torch.float, device=device)
    optimizer = optim.SGD([w], lr=1, momentum=0.9, nesterov=True)
    #optimizer = optim.Adam([w], lr=20)
    result.append(train_run(up, 10, 1000))
    plt.plot([x.item() for x in result[-1][0]], label=str(up))
plt.legend()
plt.show()

In [ ]:
tensor_to_pil(result[0][4][7].cpu())

In [ ]:
for i in range(batch):
    print(i)
    print(torch.mean(result[0][1][i][:,:,0][np.where(Y[:,:,0]>0.5)]).item())
    print(result[0][1][i][:,:,0][result[0][1][i][:,:,0]>0.5].cpu().detach().numpy())

In [ ]:
display(tensor_to_pil(result[0][2][0].repeat(1,1,912).cpu()))

In [ ]:
plt.plot(torch.flatten(result[0][3][0][0]).detach().cpu(), label="w_r")
plt.legend()
plt.show()
plt.plot(torch.flatten(result[0][3][0][1]).detach().cpu(), label="w_g")
plt.legend()
plt.show()
plt.plot(torch.flatten(result[0][3][0][2]).detach().cpu(), label="w_b")
plt.legend()
plt.show()

In [ ]:
Llp,LlpImgs,_,Y2,_ = detect_lp(wpod_net,im2single(Ivehicle),bound_dim,2**4,(240,80),lp_threshold,result[0][2][7:8])

if len(LlpImgs):
    print('found')
    Ilp = LlpImgs[0]
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_BGR2GRAY)
    Ilp = cv2.cvtColor(Ilp, cv2.COLOR_GRAY2BGR)

    s = Shape(Llp[0].pts)

    plt.imshow(Ilp)

In [ ]:
Y[...,0][np.where(Y[...,0]>0.5)]